<a href="https://colab.research.google.com/github/Nageswar-Sahoo/PyTorch_101/blob/main/Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Pytorch's tensors are similar to Numpy's ndarrays  Python Upodated Nageshwar 
!pip install torch

In [ ]:
import torch.nn.functional as F
import torch.optim as optim
import torch 
import torch
import torchvision # provide access to datasets, models, transforms, utils, etc
import torchvision.transforms as transforms
from torch.utils.data import Dataset
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import random



In [ ]:
train_set = torchvision.datasets.FashionMNIST(
    root='./data',
    train=True,
    download=True,
    transform = transforms.Compose([
          transforms.ToTensor()
    ])
)

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting ./data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting ./data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting ./data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting ./data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [ ]:
def getRandomNumber(batchsize):
  return random.sample(range(0, 10), batchsize)

In [ ]:

class Network(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5) 
    self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)
    self.fc1 = nn.Linear(in_features=232, out_features=120)
    self.fc2 = nn.Linear(in_features=120, out_features=60)
    self.out = nn.Linear(in_features=60, out_features=29)

    self.fcr1 = nn.Linear(in_features=10, out_features=20)
    self.fcr2 = nn.Linear(in_features=20, out_features=40)
  
  def forward(self, t, randomNum):
    # input layer
    x = t
    y=randomNum


    # conv1 layer
    x = self.conv1(x)
    x = F.relu(x)
    x = F.max_pool2d(x, kernel_size=2, stride=2) # 28 | 24 | 12

    # conv2 layer
    x = self.conv2(x)
    x = F.relu(x)
    x = F.max_pool2d(x, kernel_size=2, stride=2) # 12 | 8 | 4 >> 12x4x4

    # reshapre
    x = x.reshape(-1, 12 * 4 * 4)
    y = y.reshape(-1, 10)

    #print(x.shape)
    #print(y.shape)

    # fcr layer
    y = self.fcr1(y)
    y = F.relu(y)


    # fcr layer
    y = self.fcr2(y)
    y = F.relu(y)


    #print("Before concat")
    #print(x.shape)
    #print(y.shape)

    x=torch.cat((x,y ), 1)

    #print("Shape After concat ")
    #print(x.shape)


    # fc1 layer
    x = self.fc1(x)
    x = F.relu(x)

    # fc2 layer
    x = self.fc2(x)
    x = F.relu(x)




    # output layer
    x = self.out(x)
    # x = F.softmax(x, dim=1)
    return x

In [ ]:
network = Network()


In [ ]:
from torchsummary import summary

In [ ]:
summary(network, [(1,28,28),(1,10)])


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 6, 24, 24]             156
            Conv2d-2             [-1, 12, 8, 8]           1,812
            Linear-3                   [-1, 20]             220
            Linear-4                   [-1, 40]             840
            Linear-5                  [-1, 120]          27,960
            Linear-6                   [-1, 60]           7,260
            Linear-7                   [-1, 29]           1,769


/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:87: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


TypeError: ignored

In [ ]:
def get_num_correct(preds, labels):
  return preds.argmax(dim=1).eq(labels).sum().item()

In [ ]:
getRandomNumber(batch_size)

[2, 5, 8, 0, 4, 3, 1, 6, 7, 9]

In [ ]:
def getSum(list1,list2):
    return [x + y for (x, y) in zip(labels, randomNum)]

In [ ]:
torch.stack((t1,t2),dim=0)

In [ ]:
batch_size=10
train_loader = torch.utils.data.DataLoader(train_set, batch_size)
optimizer = optim.Adam(network.parameters(), lr=0.01)
sigmoid = nn.Sigmoid() # initialize sigmoid layer
lossCal = nn.BCEWithLogitsLoss() # initialize loss function

for epoch in range(1):

    total_loss = 0
    total_correct = 0

    for batch in train_loader: # Get Batch
        images, labels = batch 
        #print("One hot ")
        randomNum=getRandomNumber(batch_size)

        randomNumTensor = torch.as_tensor(randomNum)    
        res=torch.cat((F.one_hot(labels, num_classes=10), F.one_hot(randomNumTensor, num_classes=19)), 1)
        #print(randomNumTensor)
        #print(res)
        #print(labels)



        
        preds = network(images,torch.as_tensor(F.one_hot(randomNumTensor, num_classes=10) , dtype=torch.float)) # Pass Batch
        print("After Prediction")
        #print(res.type(torch.float))
        #print(sigmoid(preds))
        #print(res)
        b=res.type(torch.float)         
                
        #loss = F.cross_entropy(preds, labels) # Calculate Loss
        loss = lossCal(preds, b) # Calculate Loss

        
      
        #print(preds)
        #print(preds[:,0:10])
        print(preds[:,10:])
        #print("ARGMAX")
        #print(preds[:,0:10].argmax(dim=1))
        print(preds[:,10:].argmax(dim=1))

        #print("Actual Label")
        #print(labels)
        #print(randomNumTensor+labels)
        #print("H2")
        optimizer.zero_grad()
        loss.backward() # Calculate Gradients
        optimizer.step() # Update Weights

        total_loss += loss.item()
        total_correct += get_num_correct(preds, labels)
        break

    print(
        "epoch", epoch, 
        "total_correct:", total_correct, 
        "loss:", total_loss
    )

After Prediction
tensor([[-0.0157, -0.0581,  0.0427, -0.1088, -0.1180,  0.0925, -0.1188, -0.1093,
         -0.0489, -0.0683,  0.0252, -0.0711,  0.0197, -0.0856,  0.0713,  0.1718,
         -0.1030,  0.1284, -0.1436],
        [-0.0138, -0.0551,  0.0442, -0.1190, -0.1183,  0.1068, -0.1347, -0.1215,
         -0.0411, -0.0687,  0.0180, -0.0679,  0.0138, -0.0847,  0.0639,  0.1584,
         -0.1102,  0.1299, -0.1420],
        [-0.0101, -0.0511,  0.0391, -0.1140, -0.1247,  0.1040, -0.1176, -0.1142,
         -0.0565, -0.0586,  0.0095, -0.0513,  0.0206, -0.0785,  0.0652,  0.1580,
         -0.1050,  0.1175, -0.1389],
        [-0.0071, -0.0532,  0.0437, -0.1067, -0.1193,  0.1038, -0.1181, -0.1106,
         -0.0545, -0.0620,  0.0159, -0.0556,  0.0157, -0.0770,  0.0658,  0.1614,
         -0.1080,  0.1228, -0.1426],
        [-0.0100, -0.0539,  0.0496, -0.1098, -0.1213,  0.0957, -0.1258, -0.1079,
         -0.0501, -0.0771,  0.0251, -0.0642,  0.0208, -0.0770,  0.0724,  0.1658,
         -0.1143,  0.1310

NameError: ignored

In [ ]:
x = torch.randn(10, 5)
y = torch.randint(5, (10,))
loss = nn.CrossEntropyLoss()(x, y)

In [ ]:
x

tensor([[ 1.5424,  0.1245,  0.8532, -0.0406,  1.9486],
        [ 0.3504, -1.0312,  2.5902, -1.6936,  0.3229],
        [ 1.2855,  0.1325, -0.4814, -0.8483, -0.9243],
        [-0.6925, -0.3340, -1.0145, -0.5652,  0.3913],
        [ 0.3336,  0.6916, -1.0933, -0.2525, -1.3558],
        [-1.8370,  1.1006, -0.0550, -1.0756, -0.7686],
        [ 1.0511,  1.7066, -0.0846,  0.3604, -0.9002],
        [ 1.8079,  0.5024,  1.4377, -1.9466, -0.2282],
        [-2.7253,  0.0736,  0.1742, -0.0887, -0.5566],
        [-0.8521,  0.6604, -2.1746, -0.8742,  0.3429]])

In [ ]:
y

tensor([0, 1, 1, 2, 3, 3, 4, 1, 2, 1])

In [ ]:

#https://learnopencv.com/multi-label-image-classification-with-pytorch-image-tagging/
criterion = nn.BCELoss()


In [ ]:
def forward(self, x):
    # Do your stuff here
    ...
    x1 = F.log_softmax(x) # class probabilities
    x2 = ... # bounding box calculation
    return x1, x2



out1, out2 = model(data)
loss1 = criterion1(out1, target1)
loss2 = criterion2(out2, target2)
loss = loss1 + loss2
loss.backward()    

input:  tensor([[ 1.3642,  1.0437,  1.0348, -0.9041, -0.1930],
        [-1.2949, -2.2943,  0.0463,  0.5318, -0.2117]], requires_grad=True)
target:  tensor([0, 4])
output:  tensor(1.2981, grad_fn=<NllLossBackward>)


In [ ]:
list1=[[ 1.3642,  1.0437,  1.0348 ],
        [-1.2949, -2.2943,  0.0463,  0.5318, -0.2117, -0.9041, -0.1930]]

In [ ]:
list1

[[1.3642, 1.0437, 1.0348, -0.9041, -0.193],
 [-1.2949, -2.2943, 0.0463, 0.5318, -0.2117]]

In [ ]:

input = torch.randn(3, requires_grad=True) # give some random input
target = torch.empty(3).random_(2) # create some ground truth values
output = loss(m(input), target) # forward pass
output.backward() # backward pass

In [ ]:
output

tensor(0.5196, grad_fn=<BinaryCrossEntropyBackward>)

In [ ]:
input

tensor([ 0.7451, -0.2807,  0.9495], requires_grad=True)

In [ ]:
target

tensor([1., 1., 1.])

In [ ]:
import random
#Generate 5 random numbers between 10 and 30
randomlist1 = random.sample(range(0, 10), 5)
randomlist2 = random.sample(range(0, 10), 6)
randomlist3 = random.sample(range(0, 10), 7)
randomlist4 = random.sample(range(0, 10), 8)
print(randomlist1)
print(randomlist2)
print(randomlist3)
print(randomlist4)

[7, 8, 4, 2, 5]
[7, 6, 5, 9, 1, 4]
[3, 9, 6, 4, 7, 1, 8]
[1, 7, 4, 2, 6, 8, 5, 0]
